# Importing the Libraries


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Loading the Dataset

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [20]:
print(y)

[1 0 1 ... 1 1 0]


# Encoding Categorical Data

Label Encoding the Gender Column

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

One hot encoding the Geography column

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Splitting the dataset into the Training set and Test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Initializing the ANN. Adding the input layer, first hidden layer and output layer

In [11]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compiling and Training the ANN on Training data

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 150)

Epoch 1/150
250/250 [==============================] - 1s 3ms/step - loss: 0.3336 - accuracy: 0.8627
Epoch 2/150
250/250 [==============================] - 1s 3ms/step - loss: 0.3334 - accuracy: 0.8637
Epoch 3/150
250/250 [==============================] - 1s 3ms/step - loss: 0.3335 - accuracy: 0.8624
Epoch 4/150
250/250 [==============================] - 1s 3ms/step - loss: 0.3333 - accuracy: 0.8626
Epoch 5/150
250/250 [==============================] - 1s 3ms/step - loss: 0.3334 - accuracy: 0.8640
Epoch 6/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3336 - accuracy: 0.8621
Epoch 7/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3335 - accuracy: 0.8621
Epoch 8/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3332 - accuracy: 0.8624
Epoch 9/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3331 - accuracy: 0.8620
Epoch 10/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3330 - accura

# Making the predictions and evaluating the model

# Predicting the result of a single observation

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

In [17]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


# Predicting the Test set results

In [18]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1535   60]
 [ 204  201]]


0.868